<a href="https://colab.research.google.com/github/hfelizzola/Investigacion-Doctoral/blob/main/data_engineering_multas_sanciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Predicción de Multas y Sanciones

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
os.chdir('/gdrive/MyDrive/Colab Notebooks/Proyecto Riesgo Corrupción')

In [ ]:
!ls

consulta.xlsx				     multas
Consulta.xlsx				     obra
data_engineering_multas_sanciones.ipynb      secop_i_descarga_2020
filtrar_contratos_hospitales_publicos.ipynb  secop_ii
Indicadores-BanderasRojas.ipynb		     secop_i_match_multas
interventoria.xlsx


In [ ]:
multas = pd.read_csv('multas/multas_sanciones_secop_i.csv')
multas.head()

,Nombre Entidad,NIT Entidad,Nivel,Orden,Municipio,Numero de Resolucion,Documento Contratista,Nombre Contratista,Numero de Contrato,Valor Sancion,Fecha de Publicacion,Ruta de Proceso
0,AGENCIA NACIONAL DE INFRAESTRUCTURA ANI,830125996,NACIONAL,NACIONAL DESCENTRALIZADO,Buenaventura,RESOLUCION 822-2018 Y RESOLUCION 1284-2018,9002251332,SOCIEDAD FERROCARRIL DEL PACIFICO SAS,CONTRATO DE CONCESION No 09CONP981998,0.000000e+00,07/18/2018 12:00:00 AM,https://www.contratos.gov.co/consultas/detalle...
1,BOGOTÁ DC IDU,No Definido,TERRITORIAL,DISTRITO CAPITAL,Bogotá D.C.,14257,9002592606,CONSORCIO PRO 3,IDU932008,1.044120e+09,05/08/2014 12:00:00 AM,https://www.contratos.gov.co/consultas/detalle...
2,CASANARE GOBERNACIÓN,892099216,TERRITORIAL,TERRITORIAL DEPARTAMENTAL CENTRALIZADO,Villanueva,RESOLUCION 0064-2019 Y RESOLUCION 0070-2019,9008530116,CONSORCIO EDUCAR 2015,CTO 13212015,3.914187e+07,03/06/2019 12:00:00 AM,https://www.contratos.gov.co/consultas/detalle...
3,SANTANDER METROLINEA SA,830507387,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,Bucaramanga,035,900186817,METROCINCO PLUS SA,CONTRATO DE CONCESION UNO MLP0032007,9.856000e+05,02/05/2014 12:00:00 AM,https://www.contratos.gov.co/consultas/detalle...
4,SUPERINTENDENCIA NACIONAL DE SALUD SUPERSALUD,860062187,NACIONAL,NACIONAL DESCENTRALIZADO,Bogotá D.C.,2863,9008735825,CONSORCIO AUDIESPECIALES 2015,106 DE 2015,1.639915e+07,12/30/2015 12:00:00 AM,https://www.contratos.gov.co/consultas/detalle...


In [ ]:
"""
# Hacer el match entre secop y multas y sanciones
#!pip install fuzzymatcher
import fuzzymatcher

left_on = ["Nombre Entidad", "Nombre Contratista", "Numero de Contrato"]
right_on = ['Nombre Entidad','Nom Razon Social Contratista','Numero de Contrato']
bases_secop = ['secop_i_procesos_2014.gz',
               'secop_i_procesos_2015.gz',
               'secop_i_procesos_2016.gz',
               'secop_i_procesos_2017.gz',
               'secop_i_procesos_2018.gz',
               'secop_i_procesos_2019.gz',
               #'secop_i_procesos_2020.gz',
               #'secop_i_procesos_2021.gz',
               #'secop_i_procesos_2022.gz'
               ]

for i in bases_secop:
  # 1. Leer base de datos de secop
  secop = pd.read_csv(i, usecols=['UID','Nombre Entidad','Numero de Contrato','Modalidad de Contratacion','Tipo De Contrato','Nom Razon Social Contratista'])

  print(i + ' -> Cargado')
  # 2. Seleccionar las modalidad de contratación de interes
  #secop['Modalidad de Contratacion'].value_counts()

 #  modalidad = ['Licitación Pública',
#                'Selección Abreviada servicios de Salud',
#                'Selección Abreviada del literal h del numeral 2 del artículo 2 de la Ley 1150 de 2007',
#                'Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)',
#                'Subasta']

  modalidad = [
             'Contratación Directa (Ley 1150 de 2007)',
             'Régimen Especial'
            ]
  tipo_contrato = ['Suministro','Compraventa','Obra']

  # 3. Filtrar secop segun la modadlide de interes
  secop_merge = secop.loc[(secop['Modalidad de Contratacion'].isin(modalidad)) & 
                          (secop['Tipo De Contrato'].isin(tipo_contrato)),
                          ['UID','Nombre Entidad','Numero de Contrato','Nom Razon Social Contratista']].copy()

  # 4. Realizar el match con la base de multas y sanciones
  matched_results = fuzzymatcher.fuzzy_left_join(multas[left_on],
                                                 secop_merge,
                                                 left_on,
                                                 right_on)
  
  print(i + ' -> Match')
  
  # 5. Copiar los resultados en excel
  file_name = i[:-3] + '_match_multas_modalidad2.xlsx'
  matched_results.loc[matched_results.best_match_score > 0.5].sort_values(by='best_match_score', ascending=False).to_excel(file_name, index=False)
  print(i + ' -> Match copiado')
  """

In [ ]:
match_archivos = ['secop_i_procesos_2014_match_multas.xlsx',
                  'secop_i_procesos_2015_match_multas.xlsx',
                  'secop_i_procesos_2016_match_multas.xlsx',
                  'secop_i_procesos_2017_match_multas.xlsx',
                  'secop_i_procesos_2018_match_multas.xlsx',
                  'secop_i_procesos_2019_match_multas.xlsx']

for i in range(len(match_archivos)):
  if i == 0:
    secop_i_match = pd.read_excel('secop_i_match_multas/' + match_archivos[i])
  else:
    temp = pd.read_excel('secop_i_match_multas/' + match_archivos[i])
    secop_i_match = pd.concat([secop_i_match, temp])

In [ ]:
secop_i_match['Match'].value_counts()

no         454
si         220
revisar     69
Name: Match, dtype: int64

In [ ]:
# Contratos unicos
secop_i_match.loc[secop_i_match['Match'] == 'si']['UID'].nunique()

127

In [ ]:
"""
# Crear una base de datos con contratos de interes

#bases_secop = ['secop_i_procesos_2014.gz',
#               'secop_i_procesos_2015.gz',
#               'secop_i_procesos_2016.gz',
#               'secop_i_procesos_2017.gz',
#               'secop_i_procesos_2018.gz',
#               'secop_i_procesos_2019.gz']

bases_secop = ['secop_i_2012.gz',
               'secop_i_2013.gz'
#               'secop_i_2014.gz',
#               'secop_i_2015.gz',
#               'secop_i_2016.gz',
#               'secop_i_2017.gz',
#               'secop_i_2018.gz',
#               'secop_i_2019.gz'
               ]

#modalidad = ['Licitación Pública',
#                'Selección Abreviada servicios de Salud',
#                'Selección Abreviada del literal h del numeral 2 del artículo 2 de la Ley 1150 de 2007',
#                'Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)',
#                'Subasta']

for i in range(len(bases_secop)):
  
  if i == 0:
    secop_i = pd.read_csv('secop_i_descarga_2020/' + bases_secop[i])
    secop_i = secop_i.loc[secop_i['Tipo de Proceso'].isin(modalidad)]
  else:
    temp =  pd.read_csv('secop_i_descarga_2020/' + bases_secop[i])
    temp = temp.loc[temp['Tipo de Proceso'].isin(modalidad)]
    secop_i = pd.concat([secop_i, temp])

secop_i.to_csv('secop_i_descarga_2020/secop_i_licitacion_selabre_subasta.gz', index=False)
"""

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
"""
# Extraer contratos de obra

bases_secop = [
               'secop_i_2012.gz',
               'secop_i_2013.gz',
               'secop_i_2014.gz',
               'secop_i_2015.gz',
               'secop_i_2016.gz',
               'secop_i_2017.gz',
               'secop_i_2018.gz',
               'secop_i_2019.gz'
               ]

for i in bases_secop:
  if i == 'secop_i_2012.gz':
    secop_obra = pd.read_csv('secop_i_descarga_2020/' + i)
    secop_obra = secop_obra[(secop_obra['Tipo de Contrato'] == 'Obra')]
    print("listo " + i)
  else:
    temp = pd.read_csv('secop_i_descarga_2020/' + i)
    temp = temp[(temp['Tipo de Contrato'] == 'Obra')]
    secop_obra = pd.concat([secop_obra, temp])
    print("listo " + i)

secop_obra['Multa'] = np.int16(secop_obra['UID'].isin(secop_i_match.loc[secop_i_match['Match'] == 'si']['UID'].unique()))
secop_obra.to_csv('secop_i_descarga_2020/secop_i_obra.gz', index=False)
"""

listo secop_i_2012.gz
listo secop_i_2013.gz
listo secop_i_2014.gz
listo secop_i_2015.gz
listo secop_i_2016.gz
listo secop_i_2017.gz


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


listo secop_i_2018.gz
listo secop_i_2019.gz


In [ ]:
secop_obra.shape

(226154, 72)

In [ ]:
secop_obra['Tipo de Proceso'].value_counts()

Contratación Mínima Cuantía                                                              89045
Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)                                  45559
Régimen Especial                                                                         32339
Licitación Pública                                                                       28959
Contratación Directa (Ley 1150 de 2007)                                                  23565
Licitación obra pública                                                                   5229
Contratos y convenios con más de dos partes                                                772
Concurso de Méritos Abierto                                                                386
Selección Abreviada del literal h del numeral 2 del artículo 2 de la Ley 1150 de 2007      154
Subasta                                                                                    106
Selección Abreviada servicios de Salud            

In [ ]:
secop_i = pd.read_csv('secop_i_descarga_2020/secop_i_licitacion_selabre_subasta.gz')
secop_i.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(172383, 71)

In [ ]:
secop_i.head()

,UID,Anno Cargue SECOP,Anno Firma del Contrato,Nivel Entidad,Orden Entidad,Nombre de la Entidad,NIT de la Entidad,Código de la Entidad,ID Tipo de Proceso,Tipo de Proceso,Estado del Proceso,Causal de Otras Formas de Contratacion Directa,ID Regimen de Contratacion,Regimen de Contratacion,ID Objeto a Contratar,Objeto a Contratar,Detalle del Objeto a Contratar,Tipo de Contrato,Municipio Obtencion,Municipio Entrega,Municipios Ejecucion,Fecha de Cargue en el SECOP,Numero de Constancia,Numero de Proceso,Numero del Contrato,Cuantia Proceso,ID Grupo,Nombre Grupo,ID Familia,Nombre Familia,ID Clase,Nombre Clase,ID Ajudicacion,Tipo Identifi del Contratista,Identificacion del Contratista,Nom Raz Social Contratista,Dpto y Muni Contratista,Tipo Doc Representante Legal,Identific del Represen Legal,Nombre del Represen Legal,Fecha de Firma del Contrato,Fecha Ini Ejec Contrato,Plazo de Ejec del Contrato,Rango de Ejec del Contrato,Tiempo Adiciones en Dias,Tiempo Adiciones en Meses,Fecha Fin Ejec Contrato,Compromiso Presupuestal,Cuantia Contrato,Valor Total de Adiciones,Valor Contrato con Adiciones,Objeto del Contrato a la Firma,ID Origen de los Recursos,Origen de los Recursos,Codigo BPIN,Proponentes Seleccionados,Calificacion Definitiva,ID Sub Unidad Ejecutora,Nombre Sub Unidad Ejecutora,Ruta Proceso en SECOP I,Moneda,EsPostConflicto,Marcacion Adiciones,Posicion Rubro,Nombre Rubro,Valor Rubro,Sexo RepLegal Entidad,Pilar Acuerdo Paz,Punto Acuerdo Paz,Municipio Entidad,Departamento Entidad
0,14-1-126556-3092622,2014,2014.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,CAQUETÁ - ALCALDÍA MUNICIPIO DE LA MONTAÑITA,890680026-7,218410011,1,Licitación Pública,Liquidado,Ninguna,3.0,Estatuto General de Contratación,95000000,"Terrenos, Edificios, Estructuras y Vías",CONSTRUCCIÓN DE AULAS Y RESTAURANTES ESCOLARES...,Obra,La Montañita,La Montañita,Caquetá - La Montañita,10/01/2014,14-1-126556,LICP-003-2014,OP 007,421701953,G,"[G] Terrenos, Edificios, Estructuras y vías",0,No Definido,0,No Definido,3092622,Cédula de Ciudadanía,7.543.256 de Armenia - Quindío,HUMBERTO LOAIZA PATIÑO,Caquetá,Cédula de Ciudadanía,7.543.256 de Armenia - Quindío,HUMBERTO LOAIZA PATIÑO,11/12/2014,12/23/2014,90,D,0,0,12/23/2014,No registra,421701953,0,421701953,CONSTRUCCIÓN DE AULAS Y RESTAURANTES ESCOLARES...,0,No definido,-1,HUMBERTO LOAIZA PATIÑO C.C. No. 7.543.256 de A...,HUMBERTO LOAIZA PATIÑO C.C. No. 7.543.256 de A...,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,La Montañita,Caquetá
1,14-1-126832-3137422,2014,2014.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 4,ARAUCA - ALCALDÍA MUNICIPIO DE ARAUCA,890205334-5,281001011,1,Licitación Pública,Liquidado,Ninguna,3.0,Estatuto General de Contratación,72000000,"Servicios de Edificación, Construcción de Inst...",CONSTRUCCION DE CANALES CUNETAS ESTRUCTURAS ...,Obra,Arauca,Arauca,Arauca - Arauca,10/06/2014,14-1-126832,1L-015-2014,369-2014,1625523511,F,[F] Servicios,0,No Definido,0,No Definido,3137422,Nit de Persona Jurídica,900793119-4,CONSORCIO PORVENIR 2014,Arauca,Cédula de Ciudadanía,13.479.696 DE CUCUTA,GERARDO RAMON ARARAT COLMENARES,12/02/2014,02/03/2015,6,M,0,0,08/03/2015,No registra,1625523511,0,1625523511,CONSTRUCCION DE CANALES CUNETAS ESTRUCTURAS ...,0,No definido,-1,CONSORCIO PORVENIR 2014 RL/ GERARDO RAMON ARAR...,CONSORCIO PORVENIR 2014 RL/ GERARDO RAMON ARAR...,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,Arauca,Arauca
2,14-1-117910-3028221,2014,2014.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA - ALC...,800222498-9,288564011,1,Licitación Pública,Celebrado,Ninguna,3.0,Estatuto General de Contratación,72000000,"Servicios de Edificación, Construcción de Inst...",CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,Obra,Providencia,Providencia,"San Andrés, Providencia y Santa

In [ ]:
secop_i['Tipo de Contrato'].value_counts()

Obra                       59247
Prestación de Servicios    45182
Suministro                 38907
Compraventa                23739
Otro Tipo de Contrato       3861
Arrendamiento                578
Concesión                    277
Consultoría                  196
Fiducia                      195
Interventoría                153
Acuerdo Marco                 26
Comodato                      14
Agregación de Demanda          4
Crédito                        4
Name: Tipo de Contrato, dtype: int64

In [ ]:
#secop_i_sancionados = secop_i.loc[secop_i['UID'].isin(secop_i_match.loc[secop_i_match['Match'] == 'si']['UID'].unique())]

In [ ]:
#secop_i_sancionados

In [ ]:
secop_i['Multa'] = np.int16(secop_i['UID'].isin(secop_i_match.loc[secop_i_match['Match'] == 'si']['UID'].unique()))

In [ ]:
secop_i

,UID,Anno Cargue SECOP,Anno Firma del Contrato,Nivel Entidad,Orden Entidad,Nombre de la Entidad,NIT de la Entidad,Código de la Entidad,ID Tipo de Proceso,Tipo de Proceso,Estado del Proceso,Causal de Otras Formas de Contratacion Directa,ID Regimen de Contratacion,Regimen de Contratacion,ID Objeto a Contratar,Objeto a Contratar,Detalle del Objeto a Contratar,Tipo de Contrato,Municipio Obtencion,Municipio Entrega,Municipios Ejecucion,Fecha de Cargue en el SECOP,Numero de Constancia,Numero de Proceso,Numero del Contrato,Cuantia Proceso,ID Grupo,Nombre Grupo,ID Familia,Nombre Familia,ID Clase,Nombre Clase,ID Ajudicacion,Tipo Identifi del Contratista,Identificacion del Contratista,Nom Raz Social Contratista,Dpto y Muni Contratista,Tipo Doc Representante Legal,Identific del Represen Legal,Nombre del Represen Legal,Fecha de Firma del Contrato,Fecha Ini Ejec Contrato,Plazo de Ejec del Contrato,Rango de Ejec del Contrato,Tiempo Adiciones en Dias,Tiempo Adiciones en Meses,Fecha Fin Ejec Contrato,Compromiso Presupuestal,Cuantia Contrato,Valor Total de Adiciones,Valor Contrato con Adiciones,Objeto del Contrato a la Firma,ID Origen de los Recursos,Origen de los Recursos,Codigo BPIN,Proponentes Seleccionados,Calificacion Definitiva,ID Sub Unidad Ejecutora,Nombre Sub Unidad Ejecutora,Ruta Proceso en SECOP I,Moneda,EsPostConflicto,Marcacion Adiciones,Posicion Rubro,Nombre Rubro,Valor Rubro,Sexo RepLegal Entidad,Pilar Acuerdo Paz,Punto Acuerdo Paz,Municipio Entidad,Departamento Entidad,Multa
0,14-1-126556-3092622,2014,2014.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,CAQUETÁ - ALCALDÍA MUNICIPIO DE LA MONTAÑITA,890680026-7,218410011,1,Licitación Pública,Liquidado,Ninguna,3.0,Estatuto General de Contratación,95000000,"Terrenos, Edificios, Estructuras y Vías",CONSTRUCCIÓN DE AULAS Y RESTAURANTES ESCOLARES...,Obra,La Montañita,La Montañita,Caquetá - La Montañita,10/01/2014,14-1-126556,LICP-003-2014,OP 007,421701953,G,"[G] Terrenos, Edificios, Estructuras y vías",0,No Definido,0,No Definido,3092622,Cédula de Ciudadanía,7.543.256 de Armenia - Quindío,HUMBERTO LOAIZA PATIÑO,Caquetá,Cédula de Ciudadanía,7.543.256 de Armenia - Quindío,HUMBERTO LOAIZA PATIÑO,11/12/2014,12/23/2014,90,D,0,0,12/23/2014,No registra,421701953,0,421701953,CONSTRUCCIÓN DE AULAS Y RESTAURANTES ESCOLARES...,0,No definido,-1,HUMBERTO LOAIZA PATIÑO C.C. No. 7.543.256 de A...,HUMBERTO LOAIZA PATIÑO C.C. No. 7.543.256 de A...,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,La Montañita,Caquetá,0
1,14-1-126832-3137422,2014,2014.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 4,ARAUCA - ALCALDÍA MUNICIPIO DE ARAUCA,890205334-5,281001011,1,Licitación Pública,Liquidado,Ninguna,3.0,Estatuto General de Contratación,72000000,"Servicios de Edificación, Construcción de Inst...",CONSTRUCCION DE CANALES CUNETAS ESTRUCTURAS ...,Obra,Arauca,Arauca,Arauca - Arauca,10/06/2014,14-1-126832,1L-015-2014,369-2014,1625523511,F,[F] Servicios,0,No Definido,0,No Definido,3137422,Nit de Persona Jurídica,900793119-4,CONSORCIO PORVENIR 2014,Arauca,Cédula de Ciudadanía,13.479.696 DE CUCUTA,GERARDO RAMON ARARAT COLMENARES,12/02/2014,02/03/2015,6,M,0,0,08/03/2015,No registra,1625523511,0,1625523511,CONSTRUCCION DE CANALES CUNETAS ESTRUCTURAS ...,0,No definido,-1,CONSORCIO PORVENIR 2014 RL/ GERARDO RAMON ARAR...,CONSORCIO PORVENIR 2014 RL/ GERARDO RAMON ARAR...,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,Arauca,Arauca,0
2,14-1-117910-3028221,2014,2014.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA - ALC...,800222498-9,288564011,1,Licitación Pública,Celebrado,Ninguna,3.0,Estatuto General de Contratación,72000000,"Servicios de Edificación, Construcción de Inst...",CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,Obra,Providencia,Providencia,"San Andrés, Providenc

In [ ]:
(
  secop_i
 .groupby(['Tipo de Contrato','Multa'])
 .size()
 .unstack('Multa')
 .fillna(0)
)

Multa,0,1
Tipo de Contrato,,
Acuerdo Marco,26.0,0.0
Agregación de Demanda,4.0,0.0
Arrendamiento,578.0,0.0
Comodato,14.0,0.0
Compraventa,23721.0,18.0
Concesión,276.0,1.0
Consultoría,196.0,0.0
Crédito,4.0,0.0
Fiducia,195.0,0.0


In [ ]:
secop_i.groupby(['Estado del Proceso','Multa']).size().unstack('Multa').fillna(0)

Multa,0,1
Estado del Proceso,,
Adjudicado,5.0,0.0
Borrador,1.0,0.0
Celebrado,84616.0,67.0
Convocado,5.0,0.0
Liquidado,86672.0,59.0
Terminado sin Liquidar,958.0,0.0


In [ ]:
secop_i.groupby(['Nivel Entidad','Multa']).size().unstack('Multa').fillna(0)

Multa,0,1
Nivel Entidad,,
NACIONAL,38434.0,37.0
No Definida,1.0,0.0
TERRITORIAL,133822.0,89.0


In [ ]:
secop_i.groupby(['Orden Entidad','Multa']).size().unstack('Multa').fillna(0)

Multa,0,1
Orden Entidad,,
DISTRITO CAPITAL,5752.0,34.0
NACIONAL CENTRALIZADO,20115.0,23.0
NACIONAL DESCENTRALIZADO,19247.0,16.0
No Definido,1.0,0.0
TERRITORIAL DEPARTAMENTAL CENTRALIZADO,8899.0,19.0
TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,13497.0,18.0
TERRITORIAL DISTRITAL MUNICIPAL NIVEL 1,5933.0,1.0
TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,16093.0,3.0
TERRITORIAL DISTRITAL MUNICIPAL NIVEL 3,2580.0,0.0


In [ ]:
secop_i.groupby(['Tipo de Contrato','Multa']).size().unstack('Multa').fillna(0)

Multa,0,1
Tipo de Contrato,,
Acuerdo Marco,26.0,0.0
Agregación de Demanda,4.0,0.0
Arrendamiento,578.0,0.0
Comodato,14.0,0.0
Compraventa,23721.0,18.0
Concesión,276.0,1.0
Consultoría,196.0,0.0
Crédito,4.0,0.0
Fiducia,195.0,0.0


In [ ]:
#secop_ii = pd.read_csv('secop_ii_contratos_5_9_22.gz')

In [ ]:
#secop_ii.head()

In [ ]:
#secop_ii_procesos = pd.read_csv('secop_ii_procesos_5_9_22.gz')

In [ ]:
#secop_ii['Modalidad de Contratacion'].value_counts()

In [ ]:
#modalidades_secop_ii = ['Selección Abreviada de Menor Cuantía',
#                        'Selección abreviada subasta inversa',
#                        'Licitación pública',
#                        'Licitación pública Obra Publica',
#                        'Licitación Pública Acuerdo Marco de Precios',
#                        'Seleccion Abreviada Menor Cuantia Sin Manifestacion Interes']

#secop_ii = secop_ii.loc[secop_ii['Modalidad de Contratacion'].isin(modalidades_secop_ii)]

In [ ]:
#import fuzzymatcher

#secop_ii_merge = secop_ii[['ID Contrato','Nombre Entidad','Referencia del Contrato','Proveedor Adjudicado']].copy()

#left_on = ["Nombre Entidad", "Nombre Contratista", "Numero de Contrato"]
#right_on = ['Nombre Entidad','Proveedor Adjudicado','Referencia del Contrato']

#matched_results = fuzzymatcher.fuzzy_left_join(multas[left_on],
#                                               secop_ii_merge,
#                                               left_on,
#                                              right_on)

In [ ]:
#matched_results.loc[matched_results.best_match_score > 0].sort_values(by='best_match_score', ascending=False).to_excel('secop_ii_match_multas.xlsx', index=False)

## Tabla Maestra

Algunas ideas de variables:
- Experiencia general del contratista al momento de firmar el contrato
- Experiencia específica del contratista al momento de firmar el contrato
- Diversidad de experiencia (Calcular un indice para verificar que tanto varian 
- Multas y sanciones previas
- Porcentaje de anticipo
- Indicar si es consorcio

In [ ]:
secop_i

,UID,Anno Cargue SECOP,Anno Firma del Contrato,Nivel Entidad,Orden Entidad,Nombre de la Entidad,NIT de la Entidad,Código de la Entidad,ID Tipo de Proceso,Tipo de Proceso,Estado del Proceso,Causal de Otras Formas de Contratacion Directa,ID Regimen de Contratacion,Regimen de Contratacion,ID Objeto a Contratar,Objeto a Contratar,Detalle del Objeto a Contratar,Tipo de Contrato,Municipio Obtencion,Municipio Entrega,Municipios Ejecucion,Fecha de Cargue en el SECOP,Numero de Constancia,Numero de Proceso,Numero del Contrato,Cuantia Proceso,ID Grupo,Nombre Grupo,ID Familia,Nombre Familia,ID Clase,Nombre Clase,ID Ajudicacion,Tipo Identifi del Contratista,Identificacion del Contratista,Nom Raz Social Contratista,Dpto y Muni Contratista,Tipo Doc Representante Legal,Identific del Represen Legal,Nombre del Represen Legal,Fecha de Firma del Contrato,Fecha Ini Ejec Contrato,Plazo de Ejec del Contrato,Rango de Ejec del Contrato,Tiempo Adiciones en Dias,Tiempo Adiciones en Meses,Fecha Fin Ejec Contrato,Compromiso Presupuestal,Cuantia Contrato,Valor Total de Adiciones,Valor Contrato con Adiciones,Objeto del Contrato a la Firma,ID Origen de los Recursos,Origen de los Recursos,Codigo BPIN,Proponentes Seleccionados,Calificacion Definitiva,ID Sub Unidad Ejecutora,Nombre Sub Unidad Ejecutora,Ruta Proceso en SECOP I,Moneda,EsPostConflicto,Marcacion Adiciones,Posicion Rubro,Nombre Rubro,Valor Rubro,Sexo RepLegal Entidad,Pilar Acuerdo Paz,Punto Acuerdo Paz,Municipio Entidad,Departamento Entidad,Multa
0,14-1-126556-3092622,2014,2014.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,CAQUETÁ - ALCALDÍA MUNICIPIO DE LA MONTAÑITA,890680026-7,218410011,1,Licitación Pública,Liquidado,Ninguna,3.0,Estatuto General de Contratación,95000000,"Terrenos, Edificios, Estructuras y Vías",CONSTRUCCIÓN DE AULAS Y RESTAURANTES ESCOLARES...,Obra,La Montañita,La Montañita,Caquetá - La Montañita,10/01/2014,14-1-126556,LICP-003-2014,OP 007,421701953,G,"[G] Terrenos, Edificios, Estructuras y vías",0,No Definido,0,No Definido,3092622,Cédula de Ciudadanía,7.543.256 de Armenia - Quindío,HUMBERTO LOAIZA PATIÑO,Caquetá,Cédula de Ciudadanía,7.543.256 de Armenia - Quindío,HUMBERTO LOAIZA PATIÑO,11/12/2014,12/23/2014,90,D,0,0,12/23/2014,No registra,421701953,0,421701953,CONSTRUCCIÓN DE AULAS Y RESTAURANTES ESCOLARES...,0,No definido,-1,HUMBERTO LOAIZA PATIÑO C.C. No. 7.543.256 de A...,HUMBERTO LOAIZA PATIÑO C.C. No. 7.543.256 de A...,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,La Montañita,Caquetá,0
1,14-1-126832-3137422,2014,2014.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 4,ARAUCA - ALCALDÍA MUNICIPIO DE ARAUCA,890205334-5,281001011,1,Licitación Pública,Liquidado,Ninguna,3.0,Estatuto General de Contratación,72000000,"Servicios de Edificación, Construcción de Inst...",CONSTRUCCION DE CANALES CUNETAS ESTRUCTURAS ...,Obra,Arauca,Arauca,Arauca - Arauca,10/06/2014,14-1-126832,1L-015-2014,369-2014,1625523511,F,[F] Servicios,0,No Definido,0,No Definido,3137422,Nit de Persona Jurídica,900793119-4,CONSORCIO PORVENIR 2014,Arauca,Cédula de Ciudadanía,13.479.696 DE CUCUTA,GERARDO RAMON ARARAT COLMENARES,12/02/2014,02/03/2015,6,M,0,0,08/03/2015,No registra,1625523511,0,1625523511,CONSTRUCCION DE CANALES CUNETAS ESTRUCTURAS ...,0,No definido,-1,CONSORCIO PORVENIR 2014 RL/ GERARDO RAMON ARAR...,CONSORCIO PORVENIR 2014 RL/ GERARDO RAMON ARAR...,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,Arauca,Arauca,0
2,14-1-117910-3028221,2014,2014.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA - ALC...,800222498-9,288564011,1,Licitación Pública,Celebrado,Ninguna,3.0,Estatuto General de Contratación,72000000,"Servicios de Edificación, Construcción de Inst...",CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,Obra,Providencia,Providencia,"San Andrés, Providenc

In [ ]:
# Crear variable con el rango de la cuantia
secop_i['Rango Cuantia'] = pd.cut(secop_i['Cuantia Contrato'], 
                                  bins=[0,1000000,10000000,100000000,1000000000,1e13],
                                  labels=['Menos de 1 millon','Entre 1 y 10 millones','Entre 10 y 100 millones','Entre 100 y 1000 millones', 'Mas de 1000 millones'])

### Filtrado inicial

El propósito de este filtro es quitar contratos con las siguientes características:

- Estado del Proceso: Borrador, Convocado, Descartado
- Tipo De Contrato: Acuerdo Marco, Agregación de Demanda, Arrendamiento, Comodato, Crédito, Fiducia, Otro Tipo de Contrato
- Razon social del contratista = No Definido


In [ ]:
(
    secop_i
    .loc[secop_i['Tipo de Contrato'] == 'Obra']
    .groupby(['Objeto a Contratar','Multa'], observed=True)
    .size()
    .sort_values(ascending=False)
    #.head(10)
    #.plot(kind='barh')
    .unstack('Multa')
    .fillna(0)
 )

Multa,0,1
Objeto a Contratar,,
"Alimentos, Bebidas y Tabaco",2.0,0.0
"Artículos Domésticos, Suministros y Productos Electrónicos de Consumo",5.0,0.0
Componentes y Equipos para Distribución y Sistemas de Acondicionamiento,69.0,0.0
Componentes y Suministros Electrónicos,45.0,0.0
Componentes y Suministros de Manufactura,49.0,0.0
"Componentes y Suministros para Estructuras, Edificación, Construcción y Obras Civiles",2512.0,3.0
"Componentes, Accesorios y Suministros de Sistemas Eléctricos e Iluminación",226.0,0.0
Difusión de Tecnologías de Información y Telecomunicaciones,16.0,0.0
"Equipo Médico, Accesorios y Suministros",4.0,0.0


In [ ]:
# Filtrar por estado del proceso
estado_proceso_del = ['Borrador', 'Convocado', 'Descartado']
secop_i = secop_i.loc[-secop_i['Estado del Proceso'].isin(estado_proceso_del)]

# Filtrar por tipo de contrato
#tipo_contrato_del = ['Acuerdo Marco', 'Agregación de Demanda', 'Arrendamiento', 'Comodato', 'Crédito', 'Fiducia', 'Otro Tipo de Contrato', 'Concesión', 'Interventoría', 'Consultoría']
#secop_i = secop_i.loc[-secop_i['Tipo de Contrato'].isin(tipo_contrato_del)]

# Filtrar contratos de mas de 100 millones
secop_i = secop_i.loc[secop_i['Cuantia Contrato'] > 1e8]

In [ ]:
from xlrd import encodings
secop_i.loc[(secop_i['Tipo de Contrato'] == 'Obra')].to_excel('obra/secop_i_obra_sanciones.xlsx')

In [ ]:
secop_i.shape

(103246, 73)